In [1]:
# include your api_key and secret_key from your binance account
%run binance_keys.ipynb

In [2]:
from binance import Client

In [3]:
import pandas as pd
import time

In [4]:
client = Client(api_key, api_secret)

In [35]:
def get_top_symbol():
    df = pd.DataFrame(client.get_ticker())
    df_allusdt = df[df.symbol.str.contains('USDT')]
    df_spot_usdt = df_allusdt[~((df_allusdt.symbol.str.contains('UP')) | (df_allusdt.symbol
                                                                     .str.contains('DOWN')))]
    df_spot_usdt = df_spot_usdt[df_spot_usdt.priceChangePercent == df_spot_usdt.priceChangePercent.max()]
    return df_spot_usdt['symbol'].values[0]

In [36]:
get_top_symbol()

'ALCXUSDT'

In [37]:
def get_minute_data(symbol, interal, lookback):
    df = pd.DataFrame(client.get_historical_klines(symbol,
                                                  interal,
                                                  lookback + ' min ago UTC'))
    df = df.iloc[:,:6]
    df.columns = ['Time','Open','High','Low','Close','Volume']
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

In [38]:
var = get_minute_data(get_top_symbol(), '1m', '5')

In [28]:
def strategy(amount, SL=0.985, traget=1.02, open_position=False):
    try:
        asset = get_top_symbol()
        df = get_minute_data(asset, '1m', '60')
    except:
        time.sleep(61)
        asset = get_top_symbol()
        df = get_minute_data(asset, '1m', '60')
    quantity = round(amount/df.Close.iloc[-1])
    if (df.Close.pct_change()+1).cumprod().iloc[-1] > 1:
        buy_order = client.create_order(symbol=asset,
                                       side='BUY',
                                       type='MARKET',
                                       quantity=quantity)
        print(buy_order)
        buy_price = float(buy_order['fills'][0]['price'])
        open_position = True
        while open_position:
            try:
                df = get_minute_data(asset, '1m', '2')
            except:
                print('Something went wrong. Bot continues in 1 min')
                time.sleep(61)
                df = get_minute_data(asset, '1m', '2')
            print(f'current close is' + str(df.Close[-1]))
            print(f'current target is' + str(buy_price*target))
            print(f'current stop is' + str(buy_price*SL))
            if df.Close[-1] <= buy_price*SL or df.Close[-1] >= buy_price*Target:
                buy_order = client.create_order(symbol=asset,
                                       side='SELL',
                                       type='MARKET',
                                       quantity=quantity)
                print(order)
                break

In [ ]:
# using binance api
# while True:
#     strategy()

In [ ]:
# using web socket manager

In [29]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns = ['symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [30]:
from binance import BinanceSocketManager

In [31]:
bsm = BinanceSocketManager(client)

In [32]:
async def strategy_method_2(amount, SL=0.985, traget=1.02, open_position=False):
    try:
        asset = get_top_symbol()
        #df = get_minute_data(asset, '1m', '60')
    except:
        time.sleep(61)
        asset = get_top_symbol()
        #df = get_minute_data(asset, '1m', '60')
    socket = bsm.trade_socket(asset)
    df = get_minute_data(asset, '1m', '60')
    quantity = round(amount/df.Close.iloc[-1])
    if (df.Close.pct_change()+1).cumprod().iloc[-1] > 1:
        buy_order = client.create_order(symbol=asset,
                                       side='BUY',
                                       type='MARKET',
                                       quantity=quantity)
        print(buy_order)
        buy_price = float(buy_order['fills'][0]['price'])
        open_position = True
        while open_position:
            await socket.__aenter__()
            msg = await socket.recv()
            df = createframe(msg)
            print(f'current close is' + str(df.Close[-1]))
            print(f'current target is' + str(buy_price*target))
            print(f'current stop is' + str(buy_price*SL))
            if df.Close[-1] <= buy_price*SL or df.Close[-1] >= buy_price*Target:
                buy_order = client.create_order(symbol=asset,
                                       side='SELL',
                                       type='MARKET',
                                       quantity=quantity)
                print(order)
                break

In [ ]:
# while True:
#     await strategy_method_2()